In [1]:
# импортируем PyTorch и модуль nn
import torch
import torch.nn as nn

In [2]:
# сначала задаем гиперпараметры нашей модели

# размер контекста - # 2 слова слева, 2 справа 
# - это наше контекстное окно
CONTEXT_SIZE = 2 
# размерность матрицы эмбеддингов
EMBEDDING_DIM = 100 

In [3]:
# наш крошечный обучающий набор, разбиваем его на слова

raw_text = """To Kara's astonishment, she discovers that a portal has opened 
in her bedroom closet and two goblins have fallen through! They refuse to
return to the fairy realms and be drafted for an impending war. In an attempt
to roust the pesky creatures, Kara falls through the portal,
smack into the middle of a huge war. Kara meets Queen Selinda, who appoints
Kara as a Fairy Princess and assigns her an impossible task:
to put an end to the war using her diplomatic skills.""".split()

print(raw_text)

['To', "Kara's", 'astonishment,', 'she', 'discovers', 'that', 'a', 'portal', 'has', 'opened', 'in', 'her', 'bedroom', 'closet', 'and', 'two', 'goblins', 'have', 'fallen', 'through!', 'They', 'refuse', 'to', 'return', 'to', 'the', 'fairy', 'realms', 'and', 'be', 'drafted', 'for', 'an', 'impending', 'war.', 'In', 'an', 'attempt', 'to', 'roust', 'the', 'pesky', 'creatures,', 'Kara', 'falls', 'through', 'the', 'portal,', 'smack', 'into', 'the', 'middle', 'of', 'a', 'huge', 'war.', 'Kara', 'meets', 'Queen', 'Selinda,', 'who', 'appoints', 'Kara', 'as', 'a', 'Fairy', 'Princess', 'and', 'assigns', 'her', 'an', 'impossible', 'task:', 'to', 'put', 'an', 'end', 'to', 'the', 'war', 'using', 'her', 'diplomatic', 'skills.']


In [4]:
# создаем словарь, оставив в списке 
# только уникальные слова
vocab = set(raw_text)
vocab_size = len(vocab)

In [5]:
# смотрим словарь
print(vocab)

{'diplomatic', 'through!', 'who', 'Princess', 'portal', 'smack', 'has', 'and', 'They', 'in', 'goblins', 'return', 'have', 'Fairy', 'impending', 'an', 'meets', 'Kara', 'her', 'that', 'war', 'bedroom', 'to', 'she', "Kara's", 'portal,', 'huge', 'task:', 'end', 'a', 'To', 'be', 'astonishment,', 'falls', 'Selinda,', 'appoints', 'creatures,', 'of', 'through', 'impossible', 'fallen', 'put', 'closet', 'fairy', 'drafted', 'into', 'two', 'attempt', 'In', 'refuse', 'war.', 'middle', 'discovers', 'for', 'Queen', 'opened', 'assigns', 'using', 'the', 'realms', 'roust', 'skills.', 'pesky', 'as'}


In [6]:
# смотрим размер словаря
print(vocab_size)

64


In [7]:
# мы создаем простые отображения слов в элементы 
# – значения Python’овского словаря и наоборот
word_to_ix = {word: ix for ix, word in enumerate(vocab)}
ix_to_word = {ix: word for ix, word in enumerate(vocab)}

In [8]:
# смотрим отображения слов в элементы Python’овского словаря
print(word_to_ix)

{'diplomatic': 0, 'through!': 1, 'who': 2, 'Princess': 3, 'portal': 4, 'smack': 5, 'has': 6, 'and': 7, 'They': 8, 'in': 9, 'goblins': 10, 'return': 11, 'have': 12, 'Fairy': 13, 'impending': 14, 'an': 15, 'meets': 16, 'Kara': 17, 'her': 18, 'that': 19, 'war': 20, 'bedroom': 21, 'to': 22, 'she': 23, "Kara's": 24, 'portal,': 25, 'huge': 26, 'task:': 27, 'end': 28, 'a': 29, 'To': 30, 'be': 31, 'astonishment,': 32, 'falls': 33, 'Selinda,': 34, 'appoints': 35, 'creatures,': 36, 'of': 37, 'through': 38, 'impossible': 39, 'fallen': 40, 'put': 41, 'closet': 42, 'fairy': 43, 'drafted': 44, 'into': 45, 'two': 46, 'attempt': 47, 'In': 48, 'refuse': 49, 'war.': 50, 'middle': 51, 'discovers': 52, 'for': 53, 'Queen': 54, 'opened': 55, 'assigns': 56, 'using': 57, 'the': 58, 'realms': 59, 'roust': 60, 'skills.': 61, 'pesky': 62, 'as': 63}


In [9]:
# смотрим отображения элементов Python’овского словаря в слова
print(ix_to_word)

{0: 'diplomatic', 1: 'through!', 2: 'who', 3: 'Princess', 4: 'portal', 5: 'smack', 6: 'has', 7: 'and', 8: 'They', 9: 'in', 10: 'goblins', 11: 'return', 12: 'have', 13: 'Fairy', 14: 'impending', 15: 'an', 16: 'meets', 17: 'Kara', 18: 'her', 19: 'that', 20: 'war', 21: 'bedroom', 22: 'to', 23: 'she', 24: "Kara's", 25: 'portal,', 26: 'huge', 27: 'task:', 28: 'end', 29: 'a', 30: 'To', 31: 'be', 32: 'astonishment,', 33: 'falls', 34: 'Selinda,', 35: 'appoints', 36: 'creatures,', 37: 'of', 38: 'through', 39: 'impossible', 40: 'fallen', 41: 'put', 42: 'closet', 43: 'fairy', 44: 'drafted', 45: 'into', 46: 'two', 47: 'attempt', 48: 'In', 49: 'refuse', 50: 'war.', 51: 'middle', 52: 'discovers', 53: 'for', 54: 'Queen', 55: 'opened', 56: 'assigns', 57: 'using', 58: 'the', 59: 'realms', 60: 'roust', 61: 'skills.', 62: 'pesky', 63: 'as'}


In [10]:
# пишем функцию для создания контекстного вектора 
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

# получаем примеры для обучения, семплируя из текста - списка слов
# по контекстному окну
data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = [raw_text[i - 2], raw_text[i - 1], 
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))

In [11]:
# мы получаем список двухэлементных кортежей, первый элемент -
# переменная X (контекстное окно - 2 слова слева, 2 слова справа),
# второй элемент - зависимая переменная (интересующее нас слово),
# вы видите, что теперь мы создаем слова близко друг к другу
data

[(['To', "Kara's", 'she', 'discovers'], 'astonishment,'),
 (["Kara's", 'astonishment,', 'discovers', 'that'], 'she'),
 (['astonishment,', 'she', 'that', 'a'], 'discovers'),
 (['she', 'discovers', 'a', 'portal'], 'that'),
 (['discovers', 'that', 'portal', 'has'], 'a'),
 (['that', 'a', 'has', 'opened'], 'portal'),
 (['a', 'portal', 'opened', 'in'], 'has'),
 (['portal', 'has', 'in', 'her'], 'opened'),
 (['has', 'opened', 'her', 'bedroom'], 'in'),
 (['opened', 'in', 'bedroom', 'closet'], 'her'),
 (['in', 'her', 'closet', 'and'], 'bedroom'),
 (['her', 'bedroom', 'and', 'two'], 'closet'),
 (['bedroom', 'closet', 'two', 'goblins'], 'and'),
 (['closet', 'and', 'goblins', 'have'], 'two'),
 (['and', 'two', 'have', 'fallen'], 'goblins'),
 (['two', 'goblins', 'fallen', 'through!'], 'have'),
 (['goblins', 'have', 'through!', 'They'], 'fallen'),
 (['have', 'fallen', 'They', 'refuse'], 'through!'),
 (['fallen', 'through!', 'refuse', 'to'], 'They'),
 (['through!', 'They', 'to', 'return'], 'refuse'),
 

In [12]:
class CBOW(torch.nn.Module):
    def __init__(
        self, vocab_size, embedding_dim
    ):  # мы передаем размер словаря (vocab_size) и размерность
        # матрицы эмбеддингов (embedding_dim) в виде гиперпараметров
        super(CBOW, self).__init__()

        # вывод: 1 x embedding_dim
        # инициализируем матрицу эмбеддингов, исходя из входных данных
        # Embedding(64, 100)
        self.embeddings = nn.Embedding(
            vocab_size, embedding_dim
        )
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()

        # вывод: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim=-1)

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1, -1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.tensor([word_to_ix[word]])
        # поиск одного слова в Embedding-слое после 
        # оптимизации Embedding-слоя
        return self.embeddings(word).view(1, -1)

In [13]:
# мы инициализируем модель

model = CBOW(vocab_size, EMBEDDING_DIM)

In [14]:
# задаем функцию потерь и оптимизатор
loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

2024-03-29 17:28:50.007512: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
# обучение

for epoch in range(50):
    # начинаем отслеживать, насколько точны 
    # наши слова
    total_loss = 0

    # проходим по всем обучающим данным (x, y)
    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)

        # логвероятности
        log_probs = model(context_vector)

        # вычисляем ошибки, сравниваем фактическое целевое слово
        # с предсказанными логвероятностями
        total_loss += loss_function(log_probs, 
                                    torch.tensor([word_to_ix[target]]))
        

    # оптимизируем в конце каждой эпохи
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    # выводим некоторые метрики, чтобы увидеть, уменьшается ли ошибка
    print("конец эпохи {} | функция потерь {:2.3f}".format(epoch, total_loss))

конец эпохи 0 | функция потерь 341.410
конец эпохи 1 | функция потерь 329.548
конец эпохи 2 | функция потерь 318.385
конец эпохи 3 | функция потерь 307.745
конец эпохи 4 | функция потерь 297.539
конец эпохи 5 | функция потерь 287.680
конец эпохи 6 | функция потерь 278.142
конец эпохи 7 | функция потерь 268.879
конец эпохи 8 | функция потерь 259.886
конец эпохи 9 | функция потерь 251.095
конец эпохи 10 | функция потерь 242.463
конец эпохи 11 | функция потерь 233.953
конец эпохи 12 | функция потерь 225.584
конец эпохи 13 | функция потерь 217.299
конец эпохи 14 | функция потерь 209.169
конец эпохи 15 | функция потерь 201.178
конец эпохи 16 | функция потерь 193.297
конец эпохи 17 | функция потерь 185.538
конец эпохи 18 | функция потерь 177.900
конец эпохи 19 | функция потерь 170.370
конец эпохи 20 | функция потерь 162.943
конец эпохи 21 | функция потерь 155.625
конец эпохи 22 | функция потерь 148.443
конец эпохи 23 | функция потерь 141.413
конец эпохи 24 | функция потерь 134.521
конец эпох

In [16]:
# теперь давайте проверим, предсказывает ли модель 
# правильное слово, используя наши исходные данные
context = ["Kara", "falls", "the", "portal"]
context_vector = make_context_vector(context, word_to_ix)
# получаем вектор логвероятностей для заданного контекстного окна
a = model(context_vector)
a

tensor([[-5.9854, -4.4660, -4.7435, -5.9982, -4.9423, -4.2821, -3.9465, -5.3258,
         -5.2978, -4.2817, -4.7616, -5.1793, -5.5720, -4.9780, -6.1901, -4.5477,
         -4.5531, -2.5600, -5.3023, -3.1733, -4.8637, -5.5714, -3.5211, -5.0773,
         -6.9901, -4.6496, -4.5390, -5.1079, -5.2634, -4.1721, -6.0281, -5.1387,
         -5.1415, -3.4748, -6.1279, -5.0649, -1.8627, -5.0629, -1.7432, -5.6957,
         -5.6831, -5.7368, -4.5809, -4.7182, -4.9837, -4.8950, -5.9325, -4.9519,
         -6.2756, -5.2920, -5.1967, -3.8832, -4.1986, -5.6002, -5.3116, -3.7686,
         -5.3708, -3.8020, -3.9647, -4.4381, -4.7890, -5.2009, -3.1667, -5.0154]],
       grad_fn=<LogSoftmaxBackward0>)

In [17]:
# сопоставим логвероятности словам словаря
import numpy as np
values_list = list(ix_to_word.values())
result_dict = {word: a[0, i].item() for i, word in enumerate(values_list)}
rounded_dict = {key: round(value, 2) for key, value in result_dict.items()}
print(rounded_dict)

{'diplomatic': -5.99, 'through!': -4.47, 'who': -4.74, 'Princess': -6.0, 'portal': -4.94, 'smack': -4.28, 'has': -3.95, 'and': -5.33, 'They': -5.3, 'in': -4.28, 'goblins': -4.76, 'return': -5.18, 'have': -5.57, 'Fairy': -4.98, 'impending': -6.19, 'an': -4.55, 'meets': -4.55, 'Kara': -2.56, 'her': -5.3, 'that': -3.17, 'war': -4.86, 'bedroom': -5.57, 'to': -3.52, 'she': -5.08, "Kara's": -6.99, 'portal,': -4.65, 'huge': -4.54, 'task:': -5.11, 'end': -5.26, 'a': -4.17, 'To': -6.03, 'be': -5.14, 'astonishment,': -5.14, 'falls': -3.47, 'Selinda,': -6.13, 'appoints': -5.06, 'creatures,': -1.86, 'of': -5.06, 'through': -1.74, 'impossible': -5.7, 'fallen': -5.68, 'put': -5.74, 'closet': -4.58, 'fairy': -4.72, 'drafted': -4.98, 'into': -4.89, 'two': -5.93, 'attempt': -4.95, 'In': -6.28, 'refuse': -5.29, 'war.': -5.2, 'middle': -3.88, 'discovers': -4.2, 'for': -5.6, 'Queen': -5.31, 'opened': -3.77, 'assigns': -5.37, 'using': -3.8, 'the': -3.96, 'realms': -4.44, 'roust': -4.79, 'skills.': -5.2, 'p

In [18]:
print(f'Исходный текст: {" ".join(raw_text)}\n')
print(f"Контекст: {context}\n")
print(f"Прогноз: {ix_to_word[torch.argmax(a[0]).item()]}")

Исходный текст: To Kara's astonishment, she discovers that a portal has opened in her bedroom closet and two goblins have fallen through! They refuse to return to the fairy realms and be drafted for an impending war. In an attempt to roust the pesky creatures, Kara falls through the portal, smack into the middle of a huge war. Kara meets Queen Selinda, who appoints Kara as a Fairy Princess and assigns her an impossible task: to put an end to the war using her diplomatic skills.

Контекст: ['Kara', 'falls', 'the', 'portal']

Прогноз: through


In [19]:
# теперь давайте получим то, что нас интересует, а именно эмбеддинги
print(f"Получаем векторы для последовательностей:\n", 
      model.embeddings(torch.LongTensor([1, 2, 3])))

Получаем векторы для последовательностей:
 tensor([[-1.0090e+00, -1.2199e+00,  2.6711e-01,  1.7890e+00,  6.9306e-01,
          1.1203e+00,  8.0827e-02, -8.3560e-01, -1.2842e+00, -1.5991e-01,
         -8.3993e-01,  2.0076e+00,  9.4046e-01,  1.4451e+00,  4.4093e-01,
         -1.9838e+00,  4.8109e-01, -5.4800e-01, -1.1622e+00,  6.2339e-01,
          1.3167e-02, -6.6875e-01, -5.0148e-01,  1.5097e+00,  8.9932e-02,
         -9.6222e-01, -1.1459e+00,  5.5924e-01, -1.7697e+00, -4.0415e-01,
          4.3648e-01,  1.3770e+00,  4.3573e-01,  1.1972e+00, -1.3504e+00,
          8.5452e-01, -1.1214e-01, -9.5374e-01, -2.5653e-01, -6.9154e-01,
          1.1581e+00,  1.0974e-01,  2.6138e-01, -1.3512e+00, -4.0799e-01,
          1.8491e-01, -5.1875e-01,  1.4990e+00, -4.9743e-02, -1.2241e+00,
         -2.7252e-01,  1.9608e-01, -1.0414e+00, -6.8224e-01,  9.2100e-01,
          1.4488e+00, -9.3083e-01, -2.2348e-01,  2.2392e+00, -7.2344e-01,
          6.5876e-01,  7.4684e-01,  2.2105e+00, -5.7948e-02,  1.2236e

In [20]:
print(
    "Получаем веса:\n", model.embeddings.weight.data[1]
)  # мы можем получить всю матрицу

Получаем веса:
 tensor([-1.0090, -1.2199,  0.2671,  1.7890,  0.6931,  1.1203,  0.0808, -0.8356,
        -1.2842, -0.1599, -0.8399,  2.0076,  0.9405,  1.4451,  0.4409, -1.9838,
         0.4811, -0.5480, -1.1622,  0.6234,  0.0132, -0.6687, -0.5015,  1.5097,
         0.0899, -0.9622, -1.1459,  0.5592, -1.7697, -0.4041,  0.4365,  1.3770,
         0.4357,  1.1972, -1.3504,  0.8545, -0.1121, -0.9537, -0.2565, -0.6915,
         1.1581,  0.1097,  0.2614, -1.3512, -0.4080,  0.1849, -0.5187,  1.4990,
        -0.0497, -1.2241, -0.2725,  0.1961, -1.0414, -0.6822,  0.9210,  1.4488,
        -0.9308, -0.2235,  2.2392, -0.7234,  0.6588,  0.7468,  2.2105, -0.0579,
         1.2236,  0.4977,  0.0986,  0.1342, -2.0259, -0.4986, -0.0548,  1.2435,
         1.0405,  1.1321,  0.0110, -1.4557, -0.0565,  1.2797, -0.2269,  1.0563,
         1.1891, -0.8492,  0.6720,  0.4338,  0.8091, -0.7768,  0.1116, -1.5418,
        -0.2213, -1.2280, -0.4534, -1.3431,  1.1260,  0.5482,  0.0802,  0.9699,
        -1.3517, -0.1112

In [21]:
# а на самом деле нам важна возможность поиска 
# отдельных слов с помощью их эмбеддингов
torch.set_printoptions(threshold=10_000)
print(f"Эмбеддинг для Kara:\n{model.embeddings.weight[word_to_ix['Kara']]}")

Эмбеддинг для Kara:
tensor([ 0.1775, -0.8194, -0.1095,  0.7376,  1.6377,  0.8093,  0.6530, -0.5605,
         0.3553,  0.4160, -0.4884, -0.1703,  0.6380, -2.2777,  0.6554,  0.0107,
         1.1062, -0.1409, -1.5782, -0.2817, -0.8835, -0.1846,  0.1745,  1.1490,
         0.2203,  0.4399, -0.6069,  1.0812,  1.0867, -0.2677, -0.7717, -0.0371,
        -1.0458, -0.1202,  0.1830, -0.0294, -0.6084, -0.2098,  0.9619,  0.5723,
         0.7324, -0.4089, -1.3489, -1.3197,  1.5507,  0.1014,  0.7550, -1.1809,
         1.6273,  1.1813,  0.7977,  1.3293, -0.3964, -0.6034, -0.0787,  0.9559,
         0.8052,  0.4772, -0.6348, -0.9767,  1.9693,  0.8498,  0.4780, -1.2588,
        -0.3836, -0.7493, -1.2636,  0.6499,  1.0618,  1.6500, -0.3296, -0.7557,
        -0.7746,  0.1822,  0.3142,  0.0068,  1.0691, -1.7755, -0.2767,  0.9810,
         1.6947, -0.5697, -0.2470, -0.9102, -0.5590, -0.0069, -1.5698,  0.2673,
         0.6795,  0.0878, -0.4846,  1.6257,  1.9266,  1.1159,  0.7326,  1.1466,
         0.8172, -0.